In [1]:
import tensorflow as tf
import random
tf.set_random_seed(777)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 0.001
training_epochs = 20
batch_size = 100

In [4]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

## 1 layer

In [5]:
W = tf.Variable(tf.random_normal([784, 10]))
b = tf.Variable(tf.random_normal([10]))

hypothesis = tf.matmul(X, W) + b

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 5.766416279
Epoch: 0002 cost = 1.779114260
Epoch: 0003 cost = 1.121291371
Epoch: 0004 cost = 0.870594973
Epoch: 0005 cost = 0.737665539
Epoch: 0006 cost = 0.654213363
Epoch: 0007 cost = 0.595143469
Epoch: 0008 cost = 0.551573016
Epoch: 0009 cost = 0.517971299
Epoch: 0010 cost = 0.490173514
Epoch: 0011 cost = 0.467691229
Epoch: 0012 cost = 0.449094886
Epoch: 0013 cost = 0.432270365
Epoch: 0014 cost = 0.417922170
Epoch: 0015 cost = 0.406233643
Epoch: 0016 cost = 0.395015045
Epoch: 0017 cost = 0.385274437
Epoch: 0018 cost = 0.376552691
Epoch: 0019 cost = 0.368333774
Epoch: 0020 cost = 0.361185814


In [7]:
correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Accuracy: 0.9098


## 2 layers

In [8]:
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.Variable(tf.random_normal([256, 10]))
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [9]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 174.322270546
Epoch: 0002 cost = 41.735986391
Epoch: 0003 cost = 26.116232757
Epoch: 0004 cost = 18.114523415
Epoch: 0005 cost = 13.031405029
Epoch: 0006 cost = 9.697131536
Epoch: 0007 cost = 7.294154992
Epoch: 0008 cost = 5.388097617
Epoch: 0009 cost = 4.149328028
Epoch: 0010 cost = 3.073766001
Epoch: 0011 cost = 2.353420774
Epoch: 0012 cost = 1.698782371
Epoch: 0013 cost = 1.301953143
Epoch: 0014 cost = 1.016725462
Epoch: 0015 cost = 0.907978514
Epoch: 0016 cost = 0.663961709
Epoch: 0017 cost = 0.795447675
Epoch: 0018 cost = 0.499687943
Epoch: 0019 cost = 0.465722579
Epoch: 0020 cost = 0.455612864


In [10]:
correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Accuracy: 0.9474


## xavier_initializer & dropout

In [11]:
keep_prob = tf.placeholder(tf.float32)

In [12]:
W1 = tf.get_variable("W11", shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)

W2 = tf.get_variable("W22", shape=[256, 256], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W33", shape=[256, 10], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([10]))
hypothesis = tf.matmul(L2, W3) + b3

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feed_dict = {X: batch_xs, Y: batch_ys, keep_prob: 0.7}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

Epoch: 0001 cost = 0.478399013
Epoch: 0002 cost = 0.187814194
Epoch: 0003 cost = 0.138003246
Epoch: 0004 cost = 0.112826697
Epoch: 0005 cost = 0.094161499
Epoch: 0006 cost = 0.085518735
Epoch: 0007 cost = 0.074090952
Epoch: 0008 cost = 0.069962972
Epoch: 0009 cost = 0.061700932
Epoch: 0010 cost = 0.056443197
Epoch: 0011 cost = 0.054535835
Epoch: 0012 cost = 0.046712061
Epoch: 0013 cost = 0.044340825
Epoch: 0014 cost = 0.046506976
Epoch: 0015 cost = 0.040963898
Epoch: 0016 cost = 0.039933947
Epoch: 0017 cost = 0.035521244
Epoch: 0018 cost = 0.037394141
Epoch: 0019 cost = 0.036074139
Epoch: 0020 cost = 0.033102385


In [14]:
correct = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1}))

Accuracy: 0.9828
